# Introduction to Deep Learning in Python - Part 3

In [102]:
import pandas as pd

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

## Datasets

### Hourly Wages

In [103]:
hourly_wages_dataset = pd.read_csv('../datasets/hourly_wages.csv')

### Titanic dataset

In [104]:
titanic_train_dataset = pd.read_csv('../datasets/titanic_train.csv')
titanic_train_dataset['male'] = (titanic_train_dataset.Sex == 'male') * 1
titanic_train_dataset['age_was_missing'] = titanic_train_dataset['Age'].isnull() * 1
titanic_train_dataset.Age.fillna(titanic_train_dataset.Age.mean(), inplace=True)
titanic_train_dataset = pd.get_dummies(titanic_train_dataset, columns=['Embarked'])
titanic_train_dataset = titanic_train_dataset.drop(columns=['PassengerId', 'Sex', 'Cabin', 'Ticket', 'Sex'])

In [105]:
titanic_test_dataset = pd.read_csv('../datasets/titanic_test.csv')
titanic_test_dataset['male'] = (titanic_test_dataset.Sex == 'male') * 1
titanic_test_dataset['age_was_missing'] = titanic_test_dataset['Age'].isnull() * 1
titanic_test_dataset.Age.fillna(titanic_test_dataset.Age.mean(), inplace=True)
titanic_test_dataset = pd.get_dummies(titanic_test_dataset, columns=['Embarked'])
titanic_test_dataset = titanic_test_dataset.drop(columns=['PassengerId', 'Sex', 'Cabin', 'Ticket', 'Sex', 'Name'])

## Building deep learning models with keras

### Creating a keras model
- Sequential model: each layer has connection to one layes directly after it
- Dense layer: all nodes in previous layer connect to all nodes in current layer

### Understanding your data
You will soon start building models in Keras to predict wages based on various professional and demographic factors. Before you start building a model, it's good to understand your data by performing some exploratory analysis.

The data is pre-loaded into a pandas DataFrame called df. Use the .head() and .describe() methods in the IPython Shell for a quick overview of the DataFrame.

The target variable you'll be predicting is wage_per_hour. Some of the predictor variables are binary indicators, where a value of 1 represents True, and 0 represents False.

Of the 9 predictor variables in the DataFrame, how many are binary indicators? The min and max values as shown by .describe() will be informative here. How many binary indicator predictors are there?

**Answer: 6**

In [106]:
df = hourly_wages_dataset.copy()
df.describe()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
count,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,9.024064,0.179775,13.018727,17.822097,36.833333,0.458801,0.655431,0.292135,0.185393,0.044944
std,5.139097,0.384360,2.615373,12.379710,11.726573,0.498767,0.475673,0.455170,0.388981,0.207375
min,1.000000,0.000000,2.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.250000,0.000000,12.000000,8.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.780000,0.000000,12.000000,15.000000,35.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,11.250000,0.000000,15.000000,26.000000,44.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,44.500000,1.000000,18.000000,55.000000,64.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Specifying a model
Now you'll get to work with your first model in Keras, and will immediately be able to run more complex neural network models on larger datasets compared to the first two chapters.

To start, you'll take the skeleton of a neural network and add a hidden layer and an output layer. You'll then fit that model and see Keras do the optimization so your model continually gets better.

As a start, you'll predict workers wages based on characteristics like their industry, education and level of experience. You can find the dataset in a pandas dataframe called df. For convenience, everything in df except for the target has been converted to a NumPy matrix called predictors. The target, wage_per_hour, is available as a NumPy matrix called target.

For all exercises in this chapter, we've imported the Sequential model constructor, the Dense layer constructor, and pandas.

In [107]:
predictors = df.drop(columns=['wage_per_hour']).values
target = df['wage_per_hour'].values

# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Set up the model: model
model = Sequential()

# Add the first layer
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))

# Add the second layer
model.add(Dense(32, activation='relu'))

# Add the output layer
model.add(Dense(1))

### Compiling the model
You're now going to compile the model you specified earlier. To compile the model, you need to specify the optimizer and loss function to use. In the video, Dan mentioned that the Adam optimizer is an excellent choice. You can read more about it as well as other Keras optimizers [here](https://keras.io/optimizers/#adam), and if you are really curious to learn more, you can read the [original paper](https://arxiv.org/abs/1412.6980v8) that introduced the Adam optimizer.

In this exercise, you'll use the Adam optimizer and the mean squared error loss function. Go for it!

In [108]:
# Specify the model
n_cols = predictors.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Verify that model contains information from compiling
print("Loss function: " + model.loss)

Loss function: mean_squared_error


### Fitting the model
You're at the most fun part. You'll now fit the model. Recall that the data to be used as predictive features is loaded in a NumPy matrix called predictors and the data to be predicted is stored in a NumPy matrix called target. Your model is pre-written and it has been compiled with the code from the previous exercise.

In [109]:
# Specify the model
n_cols = predictors.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(predictors, target)

17/17 [==============================] - 0s 687us/step - loss: 64.2768


### Understanding your classification data
Now you will start modeling with a new dataset for a classification problem. This data includes information about passengers on the Titanic. You will use predictors such as age, fare and where each passenger embarked from to predict who will survive. This data is from [a tutorial on data science competitions](https://www.kaggle.com/c/titanic). Look [here](https://www.kaggle.com/c/titanic/data) for descriptions of the features.

The data is pre-loaded in a pandas DataFrame called df.

It's smart to review the maximum and minimum values of each variable to ensure the data isn't misformatted or corrupted. What was the maximum age of passengers on the Titanic? Use the .describe() method in the IPython Shell to answer this question.

**Answer: 80**

In [110]:
df = titanic_train_dataset.copy()
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,age_was_missing,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587,0.198653,0.188552,0.086420,0.722783
std,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429,0.477990,0.399210,0.391372,0.281141,0.447876
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000


### Last steps in classification models
You'll now create a classification model using the titanic dataset, which has been pre-loaded into a DataFrame called df. You'll take information about the passengers and predict which ones survived.

The predictive variables are stored in a NumPy array predictors. The target to predict is in df.survived, though you'll have to manipulate it for Keras. The number of predictive features is stored in n_cols.

Here, you'll use the 'sgd' optimizer, which stands for [Stochastic Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent). You'll learn more about this in the next chapter!

In [111]:
predictors = df.drop(columns=['Survived']).values
n_cols = predictors.shape[1]

# Convert the target to categorical: target
target = to_categorical(df.Survived)

# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(32, activation='relu', input_shape=(n_cols,)))

# Add the output layer
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(predictors, target)

28/28 [==============================] - 0s 531us/step - loss: 1.9591 - accuracy: 0.5937


### Making predictions
The trained network from your previous coding exercise is now stored as model. New data to make predictions is stored in a NumPy array as pred_data. Use model to make predictions on your new data.

In this exercise, your predictions will be probabilities, which is the most common way for data scientists to communicate their predictions to colleagues.

In [112]:
pred_data = titanic_test_dataset.copy().values

# Specify, compile, and fit the model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape = (n_cols,)))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(predictors, target)

# Calculate predictions: predictions
predictions = model.predict(pred_data)

# Calculate predicted probability of survival: predicted_prob_true
predicted_prob_true = predictions[:,1]

# print predicted_prob_true
print(predicted_prob_true)

28/28 [==============================] - 0s 531us/step - loss: 1.8303 - accuracy: 0.5668
[0.632411   0.56648326 0.7071001  0.612461   0.71608484 0.6627423
 0.6305056  0.91324437 0.5913416  0.8639076  0.59422576 0.90635407
 0.99973196 0.85957396 0.99434716 0.9070342  0.7292311  0.5611994
 0.56742567 0.58522713 0.99201185 0.51667416 0.92320365 0.9987239
 1.         0.69188136 0.9988372  0.5623874  0.94604594 0.8148543
 0.87976384 0.9262884  0.8578456  0.8822402  0.9966001  0.56767106
 0.6020126  0.6348465  0.62550086 0.99606776 0.67702764 0.91168267
 0.6089308  0.7258492  0.9860951  0.59793705 0.9359719  0.62079227
 0.9986022  0.7787571  0.99693954 0.7596654  0.8587041  1.
 0.7405453  0.93543637 0.6051318  0.59305465 0.7293489  1.
 0.61224675 0.7200862  0.6025358  0.6261446  1.         0.86092925
 0.6470355  0.9759162  0.9241262  1.         0.62808    0.5943209
 0.6060589  0.90991014 1.         1.         0.5969631  0.8657784
 0.7113208  0.62808    0.8797676  1.         0.9005366  0.5942